In [ ]:
import pandas as pd
from pathlib import Path

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from sklearn.model_selection import StratifiedKFold as SK
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score



train = pd.read_csv('../input/tabular-playground-series-dec-2021/train.csv')

test = pd.read_csv('../input/tabular-playground-series-dec-2021/test.csv')

submission = pd.read_csv('../input/tabular-playground-series-dec-2021/sample_submission.csv')


In [ ]:
def reduce_mean_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    
    for col in df.columns:
        col_type = df[col].dtypes
        
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            
            if str(col_type)[:3] =='int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min> np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    
    if verbose:
        print('Memory usage is decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem)/ start_mem))
    
    return df


In [ ]:
train = reduce_mean_usage(train)
test = reduce_mean_usage(test)


In [ ]:
train = train.sample(frac=1).reset_index(drop=True)

train_x = train.drop(['Id','Cover_Type'],axis=1)

test_x = test.drop(['Id'],axis=1)

train_y = train['Cover_Type']

train


In [ ]:
train_x.isnull().sum()


In [ ]:
train_y.isnull().sum()


In [ ]:
train_y.value_counts()


In [ ]:
drop_5_index = train_y[train_y == 5].index[0]

train_x = train_x.drop([drop_5_index],axis=0).reset_index().drop(['index'],axis=1)
train_y = train_y.drop([drop_5_index],axis=0).reset_index().drop(['index'],axis=1)


In [ ]:
train_x.describe()


In [ ]:
train_y.value_counts()


In [ ]:
scaler = MinMaxScaler()
train_df = scaler.fit_transform(train_x)
train_df = pd.DataFrame(train_df)
train_df.columns = train_x.columns
train_df


In [ ]:
test_df = pd.DataFrame(scaler.transform(test_x))
test_df.columns = test_x.columns
test_df


In [ ]:
train_df.columns


In [ ]:
train_non_dummy, train_dummy = train_df.columns[:10], train_df.columns[10:]

train_df_dummy = train_df[train_dummy]
train_df_non_dummy = train_df[train_non_dummy]


In [ ]:
test_df_dummy = test_df[train_dummy]
test_df_non_dummy = test_df[train_non_dummy]


In [ ]:
INPUT = tf.keras.layers.Input(shape = train_df_non_dummy.shape[1:], name = 'Input')
INPUT_dummy = tf.keras.layers.Input(shape = train_df_dummy.shape[1:], name = 'Input_Dummy')

dense1 = tf.keras.layers.Dense(300, activation='elu',  kernel_initializer = 'he_normal', name = 'Dense1')(INPUT)
dense2 = tf.keras.layers.Dense(300, activation='elu',  kernel_initializer = 'he_normal',name = 'Dense2')(dense1)
dense3 = tf.keras.layers.Dense(300, activation='elu',  kernel_initializer = 'he_normal',name = 'Dense3')(dense2)
dense4 = tf.keras.layers.Dense(300, activation='elu',  kernel_initializer = 'he_normal',name = 'Dense4')(dense3)
dense5 = tf.keras.layers.Dense(300, activation='elu',  kernel_initializer = 'he_normal',name = 'Dense5')(dense4)
dense6 = tf.keras.layers.Dense(300, activation='elu',  kernel_initializer = 'he_normal',name = 'Dense6')(dense5)
dense_dropout = tf.keras.layers.Dropout(0.5, name = 'Dropout')(dense6)

dummy_dense1 = tf.keras.layers.Dense(200, activation = 'elu', kernel_initializer = 'he_normal',name = 'Dummy_Dense1')(INPUT_dummy)
dummy_dense2 = tf.keras.layers.Dense(200, activation = 'elu', kernel_initializer = 'he_normal',name = 'Dummy_Dense2')(dummy_dense1)
dummy_dense3 = tf.keras.layers.Dense(200, activation = 'elu', kernel_initializer = 'he_normal',name = 'Dummy_Dense3')(dummy_dense2)
dummy_dense4 = tf.keras.layers.Dense(200, activation = 'elu', kernel_initializer = 'he_normal',name = 'Dummy_Dense4')(dummy_dense3)
dummy_dense5 = tf.keras.layers.Dense(200, activation = 'elu', kernel_initializer = 'he_normal',name = 'Dummy_Dense5')(dummy_dense4)
dummy_dense6 = tf.keras.layers.Dense(200, activation = 'elu', kernel_initializer = 'he_normal',name = 'Dummy_Dense6')(dummy_dense5)
dummy_dense_dropout = tf.keras.layers.Dropout(0.5, name = 'Dummy_Dropout')(dummy_dense6)

connect = tf.keras.layers.Concatenate(axis=1, name='Connection')([dense_dropout, dummy_dense_dropout])

connect_dense1 = tf.keras.layers.Dense(100, activation = 'elu', kernel_initializer = 'he_normal',name = 'Connect_Dense1')(connect)
connect_dense2 = tf.keras.layers.Dense(50, activation = 'elu', kernel_initializer = 'he_normal',name = 'Connect_Dense2')(connect_dense1)
connect_dense3 = tf.keras.layers.Dense(25, activation = 'elu', kernel_initializer = 'he_normal',name = 'Connect_Dense3')(connect_dense2)

OUTPUT = tf.keras.layers.Dense(6, activation = 'softmax', name = 'Output')(connect_dense3)

model = tf.keras.Model(inputs = [INPUT, INPUT_dummy], outputs = [OUTPUT], name = 'pythonash_model')


In [ ]:
model.summary()


In [ ]:
tf.keras.utils.plot_model(model, show_shapes = True, show_layer_names=True, rankdir='TB')


In [ ]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.002,
    decay_steps=5000,
    decay_rate=0.99
)
opt = tf.keras.optimizers.Adam(learning_rate = lr_schedule)
epoch_number = 20
check_pt = tf.keras.callbacks.ModelCheckpoint('pythonash_model.h5', save_best_only = True, verbose = 2)


In [ ]:
train=[]
test=[]
train_x=[]
test_x=[]

train_df=[]
test_df=[]


In [ ]:
Encoder = LabelEncoder()

y_encoded = Encoder.fit_transform(train_y)
train_y = []
y_encoded


In [ ]:
y_encoded


In [ ]:
model.compile(loss ='sparse_categorical_crossentropy', optimizer = opt, metrics = ['sparse_categorical_accuracy'])
model.fit(
x= [train_df_non_dummy, train_df_dummy], y = y_encoded, validation_split = 0.1 , batch_size = 1000, validation_batch_size = 1000,
callbacks = [check_pt], verbose =2, workers = 3, epochs = epoch_number)


In [ ]:
best_model = tf.keras.models.load_model('pythonash_model.h5')
result = best_model.predict([test_df_non_dummy, test_df_dummy])
result = Encoder.inverse_transform(np.argmax(result,axis=1))
final_result = pd.DataFrame(result)
final_result.columns =['Cover_Type']
final_result


In [ ]:
submission['Cover_Type'] = final_result['Cover_Type']

submission.to_csv('./submission.csv', index=False)

submission
